In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
import requests
from csv import DictReader
from StringIO import StringIO

def fetch_ies():
    url = 'http://web.pdc.wa.gov/MvcQuerySystem/AdvancedSearch/excel?tab=independent&year=2016&page=1&orderBy=&groupBy=&filterBy='

    headers = {
        'requester-name': 'KUOW campaign finance',
        'requester-contact': 'aepton@kuow.org',
        #'Cookie': 'ASP.NET_SessionId=nwoulf224audgtapyxcbimfz; TS01390fef=01936af35e58c8225d525f94feb74981c3c56a7147f99af108db132561bd7305d6a5467b7b1bd4e1259548c5ec6480f6720ae55811; _ga=GA1.2.1140908493.1463505845; _gat=1; __utmt=1; __utma=11181106.1140908493.1463505845.1477000114.1477329251.180; __utmb=11181106.2.10.1477329251; __utmc=11181106; __utmz=11181106.1463505856.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)'
        'Cookie': 'ASP.NET_SessionId=nwoulf224audgtapyxcbimfz; TS01390fef=01936af35e58c8225d525f94feb74981c3c56a7147f99af108db132561bd7305d6a5467b7b1bd4e1259548c5ec6480f6720ae55811; _ga=GA1.2.1140908493.1463505845; _gat=1; __utmt=1; __utma=11181106.1140908493.1463505845.1478193780.1478200183.201; __utmb=11181106.2.10.1478200183; __utmc=11181106; __utmz=11181106.1463505856.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)'
    }
    response = requests.get(url, headers=headers)
    corrected = response.text.replace(', LYNNWOOD, , WA, 98087', ', LYNNWOOD, WA, 98087')
    return [{key.strip(): row[key].strip() for key in row} for row in DictReader(StringIO(corrected))]

ies = fetch_ies()

AttributeError: 'list' object has no attribute 'strip'

In [3]:
from datetime import datetime

# Here's where date-filtering would go
cutoff = datetime.strptime('2010-10-22', '%Y-%m-%d')
filtered_ies = []
for ie in ies:
    if datetime.strptime(ie['Date'], '%Y-%m-%d') >= cutoff:
        filtered_ies.append(ie)

sponsors = {}
targets = {}

for ie in filtered_ies:
    if ie['Name'] not in targets:
        targets[ie['Name']] = {'total': 0., 'Spt': 0., 'Opp': 0., 'expenditures': []}
    targets[ie['Name']]['total'] += float(ie['Amount'])
    targets[ie['Name']][ie['SuppOpp'].title()] += float(ie['Amount'])
    targets[ie['Name']]['expenditures'].append(ie)
    
    if ie['Sponsor'] not in sponsors:
        sponsors[ie['Sponsor']] = {'total': 0., 'Spt': 0., 'Opp': 0., 'expenditures': []}
    sponsors[ie['Sponsor']]['total'] += float(ie['Amount'])
    sponsors[ie['Sponsor']][ie['SuppOpp'].title()] += float(ie['Amount'])
    sponsors[ie['Sponsor']]['expenditures'].append(ie)

NameError: name 'ies' is not defined

In [4]:
for row in sorted(sponsors, key=lambda k: sponsors[k]['total'], reverse=True):
    print row, locale.currency(sponsors[row]['total'], grouping=True)
    recipients = {}
    for exp in sponsors[row]['expenditures']:
        key = '%s - %s' % (exp['Name'], exp['SuppOpp'])
        if key not in recipients:
            recipients[key] = 0.
        recipients[key] += float(exp['Amount'])
    for recipient in sorted(recipients, key=lambda k: recipients[k], reverse=True):
        print '  ', recipient, locale.currency(recipients[recipient], grouping=True)
    print '----\n'

NameError: name 'sponsors' is not defined

In [5]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.utf8')

total_spent = 0.

target_list = ['FRANZ', 'MCLAUGHLIN']
for row in sorted(targets, key=lambda k: targets[k]['total'], reverse=True):
    if target_list:
        found = False
        for target in target_list:
            if row.startswith(target):
                found = True
        if not found:
            continue
    print row, locale.currency(targets[row]['total'], grouping=True)
    target_sponsors = {}
    supp_opp = {'SPT': 0., 'OPP': 0.}
    for exp in targets[row]['expenditures']:
        key = '%s - %s' % (exp['Sponsor'], exp['SuppOpp'])
        if key not in target_sponsors:
            target_sponsors[key] = 0.
        target_sponsors[key] += float(exp['Amount'])
        supp_opp[exp['SuppOpp']] += float(exp['Amount'])
        total_spent += float(exp['Amount'])
    print 'Supporting: %s' % locale.currency(supp_opp['SPT'], grouping=True)
    print 'Opposing: %s' % locale.currency(supp_opp['OPP'], grouping=True)
    for target_sponsor in sorted(target_sponsors, key=lambda k: target_sponsors[k], reverse=True):
        print '  ', target_sponsor, locale.currency(target_sponsors[target_sponsor], grouping=True)
    print '----\n'

print 'Total spent on IEs: %s' % locale.currency(total_spent, grouping=True)

Error: unsupported locale setting